In [1]:
!nvidia-smi

Sat Mar 20 21:58:20 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64       Driver Version: 440.64       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN RTX           Off  | 00000000:03:00.0 Off |                  N/A |
| 45%   64C    P2    72W / 280W |  22862MiB / 24219MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  TITAN RTX           Off  | 00000000:04:00.0 Off |                  N/A |
| 41%   51C    P2    58W / 280W |   3429MiB / 24220MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [2]:
import time
import numpy as np
import torch
import torchvision

import matplotlib.pyplot as plt
from scipy.special import softmax

import foolbox as fb
from torchvision import transforms

from boundary_attack import BoundaryAttack

In [3]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

In [4]:
model = torch.load('CIFAR10_ResNet50.pth', map_location=device)

In [5]:
test_tf = transforms.Compose([
    transforms.ToTensor()
])

test = torchvision.datasets.CIFAR10('./', download=True, train=False, transform=test_tf)

testloader = torch.utils.data.DataLoader(dataset=test, batch_size=256, shuffle=False)

Files already downloaded and verified


In [6]:
examples = enumerate(test)
batch_idx, (example_data, example_targets) = next(examples)

In [7]:
%%time

fmodel = fb.PyTorchModel(model, bounds=(0, 1), device=device)

treshold = 50

count = 0
i = 0
metrics = []

while count < treshold:
    
    init = i
    start = i+1
    
    target_init = test[init][1]
    target_start = test[start][1]
    
    input_or_adv = test[init][0]
    starting_points = test[start][0]
    
    input_or_adv = input_or_adv.to(device)
    starting_points = starting_points.to(device)
    
    input_or_adv = input_or_adv.reshape(1, 3, 32, 32)
    starting_points = starting_points.reshape(1, 3, 32, 32)
    
    target_init_our = int(model.forward(input_or_adv).argmax().cpu())
    target_start_our = int(model.forward(starting_points).argmax().cpu())
    
    if (target_init != target_start) and (target_init==target_init_our) and (target_start==target_start_our):
        label = torch.tensor([target_init]).to(device)
        
        print('Iteration: ', count+1)
        time.sleep(1)
        
        attack = BoundaryAttack(steps=25000, tensorboard='./logs')        
        adversarial = attack(model=fmodel, 
                             inputs=input_or_adv, 
                             starting_points=starting_points, 
                             criterion=fb.criteria.Misclassification(label), 
                             epsilons=1e-3)

        metrics_our = attack.metrics
        metrics.append(metrics_our)
        
        print('Finished: ', count+1)
        print('--------------------------------------')
        time.sleep(1)
        
        count += 1
        i += 1
        
    else:
        i += 1
        
    
np.save('metrics_original.npy', metrics)

Iteration:  1


100%|██████████| 25000/25000 [10:48<00:00, 38.54it/s]


Finished:  1
--------------------------------------
Iteration:  2


100%|██████████| 25000/25000 [10:37<00:00, 39.20it/s]


Finished:  2
--------------------------------------
Iteration:  3


...

Finished:  48
--------------------------------------
Iteration:  49


100%|██████████| 25000/25000 [10:27<00:00, 39.83it/s]


Finished:  49
--------------------------------------
Iteration:  50


100%|██████████| 25000/25000 [10:18<00:00, 40.41it/s]


Finished:  50
--------------------------------------
CPU times: user 8h 45min 22s, sys: 38min 32s, total: 9h 23min 55s
Wall time: 8h 30min 20s
